# CHSH Game

The **CHSH Game** quantum kata is a series of exercises designed
to get you familiar with the CHSH game.

In it, two players (Alice and Bob) try to win the following game:

Each of them is given a bit (Alice gets X and Bob gets Y), and
they have to return new bits (Alice returns A and Bob returns B)
so that X ∧ Y = A ⊕ B. The trick is, they can not communicate during the game.

> * ∧ is the standard bitwise AND operator.
> * ⊕ is the exclusive or, or XOR operator, so (P ⊕ Q) is true if exactly one of P and Q is true.

* You can read more about CHSH game in the [lecture notes](https://cs.uwaterloo.ca/~watrous/QC-notes/QC-notes.20.pdf) by
  John Watrous. 
* Q# Samples repository has [an implementation of the CHSH
  game](https://github.com/microsoft/Quantum/tree/main/samples/algorithms/chsh-game)
  that includes an explanation of the history and theory behind the game.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).


## Part I. Classical CHSH


### Task 1.1. Win Condition
**Input:** 

  1. Alice and Bob's starting bits (X and Y),

  2. Alice and Bob's output bits (A and B).

**Output:** 
True if Alice and Bob won the CHSH game, that is, if X ∧ Y = A ⊕ B, and false otherwise.

In [3]:
%kata T11_WinCondition 

open Microsoft.Quantum.Logical;

function WinCondition (x : Bool, y : Bool, a : Bool, b : Bool) : Bool {
    return (x and y) == Xor(a, b);
}

Success!

*Can't come up with a solution? See the explained solution in the [CHSH Game Workbook](./Workbook_CHSHGame.ipynb#Task-1.1.-Win-Condition).*

### Task 1.2. Alice and Bob's classical strategy

In this task you have to implement two functions, one for Alice's classical strategy and one for Bob's.
Note that they are covered by one test, so you have to implement both to pass the test. Once you implement one of the strategies, execute its cell - it will fail with the error message indicating that the other strategy is not implemented yet. Once you implement the second strategy, execute its cell to get the test result.

**Input:** Alice's starting bit (X).

**Output:** The bit that Alice should output (A) to maximize their chance of winning.

In [8]:
%kata T12_ClassicalStrategy 

operation AliceClassical (x : Bool) : Bool {
    return true;
}

Win rate 0.768


Success!

**Input:** Bob's starting bit (Y).

**Output:** The bit that Bob should output (B) to maximize their chance of winning.

In [7]:
%kata T12_ClassicalStrategy 

operation BobClassical (y : Bool) : Bool {
    return true;
}

Win rate 0.738


Success!

*Can't come up with a solution? See the explained solution in the [CHSH Game Workbook](./Workbook_CHSHGame.ipynb#Task-1.2.-Alice-and-Bob's-classical-strategy).*

## Part II. Quantum CHSH

In the quantum version of the game, the players still can not
communicate during the game, but they are allowed to share 
qubits from a Bell pair before the start of the game.


### Task 2.1. Entangled pair

**Input:** An array of two qubits in the $|00\rangle$ state.

**Goal:** Create a Bell state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big( |00\rangle + |11\rangle \big)$ on these qubits.

In [9]:
%kata T21_CreateEntangledPair 

operation CreateEntangledPair (qs : Qubit[]) : Unit {
    H(qs[0]);
    CNOT(qs[0], qs[1]);
}

Success!

*Can't come up with a solution? See the explained solution in the [CHSH Game Workbook](./Workbook_CHSHGame.ipynb#Task-2.1.-Entangled-pair).*

### Task 2.2. Alice's quantum strategy

**Inputs:**

  1. Alice's starting bit (X),

  2. Alice's half of Bell pair she shares with Bob.

**Goal:** Measure Alice's qubit in the Z basis if her bit is 0 (false), or the X basis if her bit is 1 (true), and return the result.
The state of the qubit after the operation does not matter.

In [13]:
%kata T22_AliceQuantum 

operation AliceQuantum (bit : Bool, qubit : Qubit) : Bool {
    if bit {
        H(qubit);
    }
    return M(qubit) == Zero ? false | true;
}

Success!

*Can't come up with a solution? See the explained solution in the [CHSH Game Workbook](./Workbook_CHSHGame.ipynb#Task-2.2.-Alice's-quantum-strategy).*

### Task 2.3. Rotate Bob's qubit

**Inputs:**

  1. The direction to rotate: true for clockwise, false for counterclockwise,

  2. Bob's qubit.

**Goal:** Rotate the qubit $\frac{\pi}{8}$ radians around the Y axis in the given direction.

<br/>
<details>
    <summary><b>Need a hint? Click here</b></summary>
  <a href="https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.ry">Ry gate</a> applies a rotation by a given angle in counterclockwise direction.
</details>

In [15]:
%kata T23_RotateBobQubit 

open Microsoft.Quantum.Math;

operation RotateBobQubit (clockwise : Bool, qubit : Qubit) : Unit {
    mutable amt = PI() / 8.0;
    if clockwise {
        set amt *= -1.0;
    }

    Ry(2.0 * amt, qubit);
}

Success!

*Can't come up with a solution? See the explained solution in the [CHSH Game Workbook](./Workbook_CHSHGame.ipynb#Task-2.3.-Rotate-Bob's-qubit).*

### Task 2.4. Bob's quantum strategy

**Inputs:**

  1. Bob's starting bit (Y),

  2. Bob's half of Bell pair he shares with Alice.

**Goal:** Measure Bob's qubit in the $\frac{\pi}{8}$ basis if his bit is 0 (false), or the $-\frac{\pi}{8}$ basis if his bit is 1 (true), and return the result.
The state of the qubit after the operation does not matter.

In [17]:
%kata T24_BobQuantum 

operation BobQuantum (bit : Bool, qubit : Qubit) : Bool {
    RotateBobQubit(not bit, qubit);
    return M(qubit) == One;
}

Success!

*Can't come up with a solution? See the explained solution in the [CHSH Game Workbook](./Workbook_CHSHGame.ipynb#Task-2.4.-Bob's-quantum-strategy).*

### Task 2.5. Play the CHSH game using the quantum strategy

**Input:**
Operations that return Alice and Bob's output bits (A and B) based on their quantum
strategies and given their respective qubits from the Bell pair.
Alice and Bob have already been told what their starting bits X and Y are.

**Goal:** Return Alice and Bob's output bits (A, B).
Note that this task uses strategies `AliceQuantum` and `BobQuantum`, which you've implemented in tasks 2.2 and 2.4, respectively.

In [19]:
%kata T25_PlayQuantumCHSH 

operation PlayQuantumCHSH (askAlice : (Qubit => Bool), askBob : (Qubit => Bool)): (Bool, Bool) {
    use (alice, bob) = (Qubit(), Qubit());
    CreateEntangledPair([alice, bob]);
    let A = askAlice(alice);
    let B = askBob(bob);
    return (A, B);
}

Success!

*Can't come up with a solution? See the explained solution in the [CHSH Game Workbook](./Workbook_CHSHGame.ipynb#Task-2.5.-Play-the-CHSH-game-using-the-quantum-strategy).*

*See the detailed discussion of the quantum strategy in the [CHSH Game Workbook](./Workbook_CHSHGame.ipynb#discussion).*